<img src="https://crowdmark-com.s3.amazonaws.com/website/images/visual-identity/crowdmark-logo-dark.png" alt="PIMS logo" width ="180" align="right">

# Exploring the Crowdmark API

[Crowdmark API Guide (draft version)](https://gist.github.com/heycarsten/46060b3cfce1eaeed325ddd3cdb79f0b)

[Draft Documentation Site](http://scattered-island.surge.sh/)

Goals: Use the Crowdmark API to
1. List all the assessments by their `id` in a given course
1. Extract student associated with a given booklet
2. Extract assessment_id from slug

In [ ]:
## Some math and data tools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [ ]:
# Import seaborn for visualizing data graphically
import seaborn as sns
sns.set()

In [ ]:
## For graphing in this notebook
%matplotlib inline

## Enter your Crowdmark API key

In [ ]:
## Execute this cell to generate a request for your API key.
## Paste your api key into the text field (and press enter); Comment out the next line to define your API Key.
# api = input("What is your Crowdmark API key:")

In [ ]:
## This cell writes the content of `api` to the file .crowdmark-key.

#with open(".crowdmark-key", "w") as text_file:
#   print(f"{api}", file=text_file)

In [ ]:
# This cell reads in the .crowdmark-key file and saves it as apiKey.
# The API key allows the computer hosting your Jupyter notebook to programmatically access data from Crowdmark.
with open(".crowdmark-key", 'r') as f:
    apiKey = f.read().rstrip()
# apiKey

## Specify the Course

In [ ]:
## SPECIFY THE COURSE YOU WISH TO INVESTIGATE HERE
# course_slug = 'demos'

course_slug ='samples'

In [ ]:
## Define the URL endpoint for accessing the API data.
cm = 'https://app.crowdmark.com'

# api_endpoint = '/api/users/{user-id}'
# api_endpoint = '/api/courses/'
# api_endpoint = '/api/courses/2015mat127-7934f/assessments'
# api_endpoint = '/api/assessments'
# api_endpoint = '/api/assessments/mat217-assignment-4/enrollments'
# api_endpoint = '/api/assessments/mat217-assignment-4/memberships'
# api_endpoint = '/api/assessments/mat217-assignment-4/questions'
# booklets_endpoint = '/api/assessments/' + assessment_slug +'/booklets'
assessments_endpoint = '/api/courses/' + course_slug + '/assessments'
# enrollments_endpoint = '/api/assessments/' + assessment_slug + '/enrollments'
# bookletsURL = cm + booklets_endpoint + '?api_key=' + apiKey
assessmentsURL = cm + assessments_endpoint + '?api_key=' + apiKey
# enrollmentsURL = cm + enrollments_endpoint + '?api_key=' + apiKey


In [ ]:
from urllib.request import urlopen

# booklets_response = urlopen(bookletsURL).read().decode('utf8')
# booklets = json.loads(booklets_response)

assessments_response = urlopen(assessmentsURL).read().decode('utf8')
assessments = json.loads(assessments_response)

# enrollments_response = urlopen(enrollmentsURL).read().decode('utf8')
# enrollments = json.loads(enrollments_response)

## Our data set has been pulled from bookletsURL and is now encoded as python dictionary in `booklets`.

## List Assessments in a Course

In [ ]:
assessments

In [ ]:
## List all the assessments in a course
for k in range(0,len(assessments['data'])):
    print(assessments['data'][k]['id'])

## Loop to merge all assessments

In [ ]:
## List all the assessments in a course
for k in range(0,len(assessments['data'])):
    print(assessments['data'][k]['id'])

In [ ]:
## Configure the first data frame
url0 = 'https://app.crowdmark.com/api/assessments/' + assessments['data'][0]['id'] + '/results.csv' + '?api_key=' + apiKey
dgd = pd.read_csv(url0)
## Ingore some columns
dgd = dgd.drop(['Crowdmark ID', 'Score URL', 
                 'Total', 'Penalty', 'Total After Penalty','Submitted At', 
                'Username', 'First Name', 'Last Name', 
                'Canvas ID', 'Email.1', 'Name', 'Student ID', 
                'first_name', 'full_name', 'id', 'last_name'], axis=1, errors='ignore')

In [ ]:
dgd.head(n=12)

In [ ]:
dgd.columns

In [ ]:
## Merge scores from all the assessments in the course into a single data frame.
for k in range(1,len(assessments['data'])):
## for k in range(1,2):
    print(assessments['data'][k]['id'])
    assess_slug = assessments['data'][k]['id']
    #URL for API, Crowdmark API URL in this case
    url1 = 'https://app.crowdmark.com/api/assessments/' + assess_slug + '/results.csv' + '?api_key=' + apiKey
    dg1 = pd.read_csv(url1)
    dg1 = dg1.drop(['Crowdmark ID', 'Score URL', 
                 'Total', 'Penalty', 'Total After Penalty','Submitted At', 
                'Username', 'First Name', 'Last Name', 
                'Canvas ID', 'Email.1', 'Name', 'Student ID', 
                'first_name', 'full_name', 'id', 'last_name'], axis=1, errors='ignore')
    dgd = dgd.merge(dg1, left_on=['Email'], 
                 right_on=['Email'], how='outer')

In [ ]:
dgd.columns.max

## Analyze the Scores in the Class

In [ ]:
## Cut and Paste the Question Score Columns
dgda = dgd[['Q1_x', 'Q2_x', 'Q3_x', 'Q4_x', 'Q5_x', 'Q6_x', 'Q1_y', 'Q2_y',
       'Q3_y', 'Q4_y', 'Q5_y', 'Q6_y', 'Q7', 'Q1_x', 'Q2_x', 'Q3_x', 'Q1_x',
       'Q2_y', 'Q3_y', 'Q1_y', 'Q2_x', 'Q3_x', 'Q1_x', 'Q2_y', 'Q3_y', 'Q1_y',
       'Q2_x', 'Q3_x', 'Q1_x', 'Q2_y', 'Q3_y','Q1_y', 'Q2_x', 'Q3_x', 'Q1_x',
       'Q2_y', 'Q3_y', 'Q1_y', 'Q2_x', 'Q3_x','Q1', 'Q2_y', 'Q3_y']]

In [ ]:
dgda

In [ ]:
corr = dgda.corr

In [ ]:
plt.figure(figsize = (15,12))
ax = sns.heatmap(corr('pearson'), cmap="viridis")

In [ ]:
len(dgda)